# Create samples

In [ ]:
import os
from tqdm import tqdm
from utils.utils_data import create_distortions_batch

In [ ]:
folder_path = "../datas/temp"
files = [f for f in os.listdir(folder_path) if not f.startswith('.') and '__' not in f]

In [ ]:
with tqdm(total=len(files), desc="Images to Distort") as pbar:
    for i, f in enumerate(files):
        img_path = os.path.join(folder_path, f)
        create_distortions_batch(img_path, folder_path, batch_size=1, counter=i)
        pbar.update(1)
        # if i == 2:
        #     break